In [1]:
!pip install awswrangler

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 1.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.8/220.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.1/112.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.5/131.5 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.4 MB/s eta 0:00:00


In [26]:
import boto3
import awswrangler as wr
import warnings
warnings.filterwarnings("ignore")

In [7]:
#load the AWS credentials and keep it ready

import configparser

reader = configparser.ConfigParser()
reader.read_file(open('/kaggle/input/private-set/calter.config'))


aws_reg = reader["AWS"]["REGION"]
aws_key = reader["AWS"]["KEY"]
aws_sec = reader["AWS"]["SECRET"]

my_session = boto3.Session(aws_access_key_id=aws_key,aws_secret_access_key=aws_sec,
                          region_name=aws_reg)

s3_client = boto3.client('s3',region_name=aws_reg,aws_access_key_id=aws_key,
                        aws_secret_access_key=aws_sec)

There are many sources from which the data can come to Data Engineer or the Analyst. How to pull the data from these sources reliably? 

1 - List the sources and the designate a staging area. 

2 - Think of the source Type and the connection method.

3 - Get the necessary credentials and register them for use.

4 - Code the functions that pull the data into the staging.

5 - Rinse and Repeat

### Lets Designate a Staging Area:

Stage @ AWS ==> s3://pipe-line-source

Place the string path in the variable

In [22]:
stage_path = "pipe-line-source"

In [25]:
# There are two ways to list the contents of the bucket

#Boto way: Note the bucket name is not having the prefix s3://
try:
    s3_client.list_objects_v2(Bucket=stage_path)["Contents"]
except Exception as e:
    print("The bucket must be empty.")

The bucket must be empty.


In [28]:
# listing the buckets

wr.s3.list_buckets(boto3_session=my_session)

['athena-query-res-jan',
 'aws-athena-query-results-642924624251-us-east-1',
 'aws-glue-assets-642924624251-us-east-1',
 'boto-bucket-16',
 'de-yt-starterdata-bkp',
 'pipe-line-source',
 'tab-mcq-de']

In [27]:
#AWSWrangler way

wr.s3.list_objects(path=f's3://'+stage_path,boto3_session=my_session)

#AWSWrangler is quiet when it comes to empty buckets. 

[]

List of Sources : 

In this notebook we will consider the following sources

1) CSV or Json file from Kaggle dataset 
    
    Use Kaggle notebook to load the dataset and then upload to S3
    
2) File of any type in the local file system:

    The process will be same as the above steps used in Kaggle Notebook. 

3) Data needs to be pulled from a web API 

4) Tables from Local or Remote Relational Database needs to be dumped 

5) Tables inside Data Warehouses

### Loading files from Kaggle

a) Kaggle Notebook

In [29]:
%%sh
cd /kaggle/input/bosch-production-line-performance
ls

sample_submission.csv.zip
test_categorical.csv.zip
test_date.csv.zip
test_numeric.csv.zip
train_categorical.csv.zip
train_date.csv.zip
train_numeric.csv.zip


In [32]:
# Get the file paths that needs to be uploaded

import os, json, glob
file_path = glob.glob(pathname="/kaggle/input/bosch-production-line-performance/*.zip",
                      recursive=True)
file_path

['/kaggle/input/bosch-production-line-performance/train_date.csv.zip',
 '/kaggle/input/bosch-production-line-performance/sample_submission.csv.zip',
 '/kaggle/input/bosch-production-line-performance/train_numeric.csv.zip',
 '/kaggle/input/bosch-production-line-performance/test_date.csv.zip',
 '/kaggle/input/bosch-production-line-performance/test_categorical.csv.zip',
 '/kaggle/input/bosch-production-line-performance/test_numeric.csv.zip',
 '/kaggle/input/bosch-production-line-performance/train_categorical.csv.zip']

In [36]:
#Pumping the files into S3
for fqp in file_path:
    key = fqp.split('/')[4]
    print(key)
    wr.s3.upload(local_file=fqp,path=f's3://{stage_path}/{key}',
                     boto3_session=my_session)

train_date.csv.zip


Exception ignored in: <function _S3ObjectBase.__del__ at 0x7f53e2afd200>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/awswrangler/s3/_fs.py", line 243, in __del__
    self.close()
  File "/opt/conda/lib/python3.7/site-packages/awswrangler/s3/_fs.py", line 483, in close
    function_name="put_object", s3_additional_kwargs=self._s3_additional_kwargs
  File "/opt/conda/lib/python3.7/site-packages/awswrangler/_utils.py", line 348, in try_it
    return f(**kwargs)
  File "/opt/conda/lib/python3.7/site-packages/botocore/client.py", line 530, in _api_call
    return self._make_api_call(operation_name, kwargs)
  File "/opt/conda/lib/python3.7/site-packages/botocore/client.py", line 944, in _make_api_call
    operation_model, request_dict, request_context
  File "/opt/conda/lib/python3.7/site-packages/botocore/client.py", line 966, in _make_request
    return self._endpoint.make_request(operation_model, request_dict)
  File "/opt/conda/lib/python3.7/site-pac

sample_submission.csv.zip
train_numeric.csv.zip
test_date.csv.zip
test_categorical.csv.zip
test_numeric.csv.zip
train_categorical.csv.zip


In [40]:
#Validate the files are inside the bucket
wr.s3.list_objects(path=f's3://{stage_path}',boto3_session=my_session)

['s3://pipe-line-source/bosch-production-line-performance',
 's3://pipe-line-source/sample_submission.csv.zip',
 's3://pipe-line-source/test_categorical.csv.zip',
 's3://pipe-line-source/test_date.csv.zip',
 's3://pipe-line-source/test_numeric.csv.zip',
 's3://pipe-line-source/train_categorical.csv.zip',
 's3://pipe-line-source/train_date.csv.zip',
 's3://pipe-line-source/train_numeric.csv.zip']

In [41]:
#Deleting the files in the bucket
wr.s3.delete_objects(path=f's3://{stage_path}',boto3_session=my_session)

In [42]:
#validate deletion
wr.s3.list_objects(path=f's3://{stage_path}',boto3_session=my_session)

[]

### Loading files from Local Filesystem

local_path = 'your/local/file/system/path'

Only change is the local_path name. Rest all remains same.

We will see how to put_objects using the Boto3 way...

In [49]:
#uploading the objects using boto3 client. 

#Ensure the files that needs to uploaded are opened, read and uploaded. Else only the name
#will be written

for fqp in file_path:
    key = fqp.split('/')[4]
    print(key)
    with open(fqp,mode='rb') as temp_bytes:
        s3_client.put_object(Body=temp_bytes,Bucket=stage_path,
                            Key=key)

train_date.csv.zip
sample_submission.csv.zip
train_numeric.csv.zip
test_date.csv.zip
test_categorical.csv.zip
test_numeric.csv.zip
train_categorical.csv.zip


In [53]:
#Validate the buckets the boto3 way and get their keys
#Prepare the Delete Key Objects
file_keys = [file['Key'] for file in s3_client.list_objects_v2(Bucket=stage_path)["Contents"]]

In [54]:
#Validate the file_keys
file_keys

['sample_submission.csv.zip',
 'test_categorical.csv.zip',
 'test_date.csv.zip',
 'test_numeric.csv.zip',
 'train_categorical.csv.zip',
 'train_date.csv.zip',
 'train_numeric.csv.zip']

In [55]:
#Assemble the objects necessary for sending the deleting request
file_objects = []
for keys in file_keys:
    temp = {"Key":keys}
    file_objects.append(temp)
file_objects

[{'Key': 'sample_submission.csv.zip'},
 {'Key': 'test_categorical.csv.zip'},
 {'Key': 'test_date.csv.zip'},
 {'Key': 'test_numeric.csv.zip'},
 {'Key': 'train_categorical.csv.zip'},
 {'Key': 'train_date.csv.zip'},
 {'Key': 'train_numeric.csv.zip'}]

In [56]:
#Delete the contents of the buckets Boto3 way
s3_client.delete_objects(Bucket=stage_path,
                         Delete={'Objects':file_objects,
                                 'Quiet': True})

{'ResponseMetadata': {'RequestId': 'ZTCFDN82W051QTD8',
  'HostId': '/kc9KZzKFMVQh8xaxNBLQfg4eNCee7jSqGj1VgU05eRYkkLqopDgO9Bat3ITP6gc+n5pVKzBpGc=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '/kc9KZzKFMVQh8xaxNBLQfg4eNCee7jSqGj1VgU05eRYkkLqopDgO9Bat3ITP6gc+n5pVKzBpGc=',
   'x-amz-request-id': 'ZTCFDN82W051QTD8',
   'date': 'Sat, 28 Jan 2023 04:20:29 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3',
   'connection': 'close'},
  'RetryAttempts': 0}}